<a href="https://colab.research.google.com/github/soumyadip-Roy/BLE_Module/blob/main/ble_collab_build_apk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kivy pyjnius buildozer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 36.6 MB/s eta 0:00:00


# Building the Pyjnius application


## New Application Bluetooth Module


In [ ]:
from jnius import autoclass, PythonJavaClass, java_method, java_field

import os

BluetoothAdapter = autoclass('android.bluetooth.BluetoothAdapter')
PythonActivity = autoclass('org.kivy.android.PythonActivity')

class BLEScanner(PythonJavaClass):
  __javainterfaces__ = ['android/bluetooth/le/ScanCallback']

  def __inti__(self, on_found):
    super().__init__()
    self.on_found = on_found

  @java_method('(Landroid/bluetooth/le/ScanResult;)V')
  def onScanResult(self,callbackType, result):
    device=result.getDevice()
    name = device.getName() or "Unknown"
    address = device.getAddress()
    self.on_found(name, address)

class BluetoothManager:
  def __init__(self,on_found):
    self.adapter = BluetoothAdapter.getDefaultAdapter()
    self.scanner = self.adapter.getBluetoothLeScanner()
    self.callback = BLEScanner(on_found)

  def start_scan(self):
    self.scanner.startScan(self.callback)

  def stop_scan(self):
    self.scanner.stopScan(self.callback)

  def connect_to_device(self, address):
    device = self.adapter.getRemoteDevice(address)
    device.connectGatt(PythonActivity.mActivity, False, BluetoothGattCallback())

class BluetoothGattCallback(PythonJavaClass):
  __javainterfaces__=['android/bluetooth/BluetoothGattCallback']

  @java_method('(Landroid/bluetooth/BluetoothGatt;II)V')
  def onConnectionStateChange(self, gatt, status, newState):
    if newState == 2:
      print('Connection state changed to ', newState, " connected!")


